In [1]:
import csv

# Open the CSV file for reading
with open('/content/sample_data/Higher_order_Lower_order.csv', 'r') as csvfile:
    
    # Create a CSV reader object
    csvreader = csv.DictReader(csvfile)
    
    # Create an empty list to store the data
    data = []
    
    # Loop through each row in the CSV file
    for row in csvreader:
        
        # Add the row as a dictionary to the list
        data.append(row)


In [55]:
# Create an empty list to store the new data
text = []
label = []

# Loop through each example in the original data
for example in data:
    
    # Get the text and label of the example
    text_data = example['\ufefftext']
    label_data = example['label']
    
    # Add the text and label as a tuple to the new data list
    text.append(text_data)
    label.append(int(label_data))

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.0 MB/s eta 0:00:00


In [77]:
df

,text,label
0,What is the difference between erosion and wea...,0
1,How does natural selection contribute to the e...,1
2,What is the impact of technological advancemen...,1
3,How does the structure of the brain influence ...,1
4,What are the implications of biotechnology in ...,1
...,...,...
82,What are the types of chemical reactions?,0
83,What are the characteristics of mixtures and s...,0
84,How does the process of DNA replication contri...,1
85,What is the impact of biotechnology on human h...,1


In [76]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Load data from CSV file
df = pd.read_csv('/content/sample_data/Higher_order_Lower_order.csv')
# Shuffle the rows
df = df.sample(frac=1).reset_index(drop=True)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.2)

# Load BERT tokenizer and encode data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

train_labels = torch.tensor(train_labels.tolist())
val_labels = torch.tensor(val_labels.tolist())

# Create BERT model and set to training mode
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.train()

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)

# Train model
for i in range(0, len(train_encodings["input_ids"]), 32):
    batch_encodings = {
        key: torch.tensor(val[i:i+32]) for key, val in train_encodings.items()
    }
    batch_labels = torch.tensor(train_labels[i:i+32])
    outputs = model(**batch_encodings, labels=batch_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


model.eval()

# initialize variables to accumulate the validation accuracy and loss
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

# loop through the validation dataset in batches of 32
for i in range(0, len(val_encodings["input_ids"]), 32):
    # prepare the batch data
    batch_encodings = {
        key: torch.tensor(val_encodings[key][i:i+32]) for key in val_encodings
    }
    batch_labels = torch.tensor(val_labels[i:i+32])
    
    # disable gradient calculation to speed up the inference
    with torch.no_grad():
        # perform a forward pass through the model
        outputs = model(**batch_encodings, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

    # accumulate the validation loss
    total_eval_loss += loss.item()

    # convert logits to probabilities and get the predicted label for each instance
    probs = torch.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1)

    # accumulate the validation accuracy
    total_eval_accuracy += torch.sum(preds == batch_labels).item()

    # accumulate the number of batches
    nb_eval_steps += 1

# calculate the average validation accuracy and loss
avg_val_accuracy = total_eval_accuracy / len(val_labels)
avg_val_loss = total_eval_loss / nb_eval_steps

print(f"Validation Accuracy: {avg_val_accuracy:.2f}")
print(f"Validation Loss: {avg_val_loss:.2f}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Validation Accuracy: 0.44
Validation Loss: 0.65


In [80]:
# Example random text
text = "What is the relationship between Earth's geologic history and the formation of fossil fuels?"

# Encode the text using the tokenizer
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")

# Pass the input through the model to get the predicted label
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs).item()

# Print the predicted label
print(f"Predicted label: {pred_label}")


Predicted label: 1


In [81]:
# Example random text
text = "What is the difference between velocity and acceleration?"

# Encode the text using the tokenizer
inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")

# Pass the input through the model to get the predicted label
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs).item()

# Print the predicted label
print(f"Predicted label: {pred_label}")

Predicted label: 0


In [82]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

# Load data from CSV file
df = pd.read_csv('/content/sample_data/Higher_order_Lower_order.csv')

# Shuffle the rows
df = df.sample(frac=1).reset_index(drop=True)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['label'], test_size=0.2)

# Load scientific BERT tokenizer and encode data
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

train_labels = torch.tensor(train_labels.tolist())
val_labels = torch.tensor(val_labels.tolist())

# Create scientific BERT model and set to training mode
model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=2)
model.train()

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)

# Train model for 3 epochs
for epoch in range(3):
    print(f"Epoch {epoch + 1}/3")
    for i in range(0, len(train_encodings["input_ids"]), 32):
        batch_encodings = {
            key: torch.tensor(val[i:i+32]) for key, val in train_encodings.items()
        }
        batch_labels = torch.tensor(train_labels[i:i+32])
        outputs = model(**batch_encodings, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Evaluate model after each epoch
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for i in range(0, len(val_encodings["input_ids"]), 32):
        batch_encodings = {
            key: torch.tensor(val_encodings[key][i:i+32]) for key in val_encodings
        }
        batch_labels = torch.tensor(val_labels[i:i+32])
        with torch.no_grad():
            outputs = model(**batch_encodings, labels=batch_labels)
            loss = outputs.loss
            logits = outputs.logits
        total_eval_loss += loss.item()
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        total_eval_accuracy += torch.sum(preds == batch_labels).item()
        nb_eval_steps += 1
    avg_val_accuracy = total_eval_accuracy / len(val_labels)
    avg_val_loss = total_eval_loss / nb_eval_steps
    print(f"Validation Accuracy: {avg_val_accuracy:.2f}")
    print(f"Validation Loss: {avg_val_loss:.2f}")
    model.train()


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

Epoch 1/3


<ipython-input-82-64edc4ae3743>:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_labels = torch.tensor(val_labels[i:i+32])


Validation Accuracy: 0.83
Validation Loss: 0.46
Epoch 2/3
Validation Accuracy: 0.94
Validation Loss: 0.28
Epoch 3/3
Validation Accuracy: 0.89
Validation Loss: 0.23


In [89]:
# Load BERT tokenizer and encode text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
text = "What is the difference between velocity and acceleration?"
encodings = tokenizer(text, truncation=True, padding=True, return_tensors='pt')

# Pass the encodings to the model for inference
with torch.no_grad():
    outputs = model(**encodings)
    logits = outputs.logits

# Convert logits to probabilities and get the predicted label
probs = torch.softmax(logits, dim=1)
pred_label = torch.argmax(probs, dim=1)

print(f"Predicted label: {pred_label.item()}")


Predicted label: 0
